In [1]:
from langchain_community.llms import GooglePalm
import configparser

In [5]:
config = configparser.RawConfigParser()
config.read('../config.config')
googleapi_key = config.get('Keys', 'googleapi_key')


In [6]:
llm = GooglePalm(google_api_key = googleapi_key, temperature = 0.7 )

In [7]:
poem = llm("Write a 4 line poem for my love")
print(poem)

C:\Users\Vignesh_B\Documents\GitHub\Understanding_LLMs\v_env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Your love is like a river,
Flowing through my heart.
It fills me with joy and peace,
And makes me whole.


In [8]:
essay = llm("Write an email requesting sick leave")
print(essay)

Dear [Boss's name],

I am writing to request sick leave for [date] through [date]. I have been experiencing [symptoms] and have seen a doctor, who has diagnosed me with [illness]. I am expected to be out of work for [number of days] days.

I apologize for any inconvenience this may cause. I have already completed all of my outstanding work, and I have asked [coworker's name] to cover my responsibilities while I am out.

I will keep you updated on my progress and will return to work as soon as I am able.

Thank you for your understanding.

Sincerely,
[Your name]


In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='qna.csv', source_column='prompt')
data = loader.load()

In [10]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
embed = instructor_embeddings.embed_query('What is you refund policy?')

load INSTRUCTOR_Transformer
max_seq_length  512


In [15]:
from langchain_community.vectorstores import FAISS

vectordb = FAISS.from_documents(documents =data, embedding=instructor_embeddings)
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [16]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

In [17]:
# from langchain_community.embeddings import GooglePalmEmbeddings
# google_palm_embeddings = GooglePalmEmbeddings(google_api_key=googleapi_key)
# from langchain.vectorstores import Chroma

# vectordb = Chroma.from_documents(data,
#                            embedding=google_palm_embeddings,
#                            persist_directory='./chromadb')
# vectordb.persist()

In [24]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [25]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [23]:
chain("Is EMI payments  and internships available here?")

C:\Users\Vignesh_B\Documents\GitHub\Understanding_LLMs\v_env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'Is EMI payments  and internships available here?',
 'result': 'No, EMI payments are not available. Yes, virtual internships are available.',
 'source_documents': [Document(page_content='prompt: Do we have an EMI option?\nresponse: No', metadata={'source': 'Do we have an EMI option?', 'row': 13}),
  Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
  Document(page_content='prompt: Is this bootcamp enough for me in Microsoft Power BI and\n Excel certifications?\nresponse: Yes, this bootcamp will certainly help because we cover the majority of the skills measured in these exams. However, please be informed that this course focuses on Job ready aspects and not on all aspects required to clear the exams. In addition to this course, you might need to visit the official learning material designed by Microsoft which is available for free on their website.', metadata={'sour